Do Imports

In [1]:
# This is a pre requirement to the 'pattern framework below'
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/chex/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [145]:
from pattern.web import Twitter
from pattern.text.en import tag
from pattern.vector import KNN, count
from tqdm import tqdm
import pandas as pd


In [3]:
# Test loading the dataset
df = pd.read_json('./datasets/renttherunway_final_data.json')

In [215]:
# seperate all the good reviews from the bad reviews
# get a list of adjectives of the good reviews and train those to be 'Positive'
# get a list of adjectives of the bad reviews and train those to be 'Negative'

good_df = []
bad_df = []

# TODO: Need to split this data set into 70 / 15 / 15 set for training / validation / test
#for index, row in df.iterrows():
for index, row in tqdm(df[0:10000].iterrows()):
    if row['rating'] >= 9:
        good_df.append(row)
    elif row['rating'] <= 2:
        bad_df.append(row)
    else:
        continue
    
print(f"Amount of positive reviews: {len(good_df)}")
print(f"Amount of negative reviews: {len(bad_df)}")
good_result = pd.concat(good_df)
bad_result = pd.concat(bad_df)

# generate list of good words
good_words = []
bad_words = []

print("Computing list of positive words...")
for item in good_result['review_text']:
    v = tag(item)
    v = [word for word, pos in v if pos == 'JJ'] # JJ = adjective
    v = count(v) # {'sweet': 1}
    good_words += list(v.keys())
    
print("Computing list of negative words...")
for item in bad_result['review_text']:
    v = tag(item)
    v = [word for word, pos in v if pos == 'JJ']
    v = count(v)
    bad_words += list(v.keys())

print("Done")
# print(result.shape[0])

10000it [00:00, 16970.26it/s]


Amount of positive words: 6458
Amount of negative words: 46
Computing list of positive words...
Computing list of negative words...
Done


In [216]:
print(f"Total amount of positive words: {len(good_words)}")
print(f"Total amount of negative words: {len(bad_words)}")

Total amount of positive words: 28867
Total amount of negative words: 135


TODO: Change this with the main review text from above
Look into ways to classify the customer sentiment
Experiment how much you need to change the dataset by in order to change sentiment

Further:
Look into and document this model and how it operates
It's type, size etc to help with report writing

training on bad words, while the rest is deemed positive
can look to reerse this and compare

In [220]:
# train the model on dataset

twitter, knn = Twitter(), KNN()

bad_words_final = list(set(bad_words))
good_words_final = list(set(good_words))
# print(bad_words_final)

# issue here is that the bad_list files have positive words - need to filter these out and seperate them from the 
# good words in order to train the model accurately
# OR fix/verify the below list comprehension method?

bad_words_final_filtered = [i for i in bad_words_final if i not in good_words_final]

print("Length of complete bad list", len(bad_words_final))
print("Length of filtered bad list", len(bad_words_final_filtered))

print("Test non filtered", bad_words_final)
print("Test filtered", bad_words_final_filtered)

for text in tqdm(df['review_text'][0:1000]):
    v = tag(text)
    v = [word for word, pos in v if pos == 'JJ'] # JJ = adjective
    v = count(v) # {'sweet': 1}
    
#     print("list of adjectives to compare against", list(v.keys()))
    if any(word in bad_words_final_filtered for word in list(v.keys())):
        p = 'Bad'
    else:
        p = 'Good'
    
#     print(p)
#     print("---")
    if v:
        knn.train(v, type=p)

print("Training done")

Length of complete bad list 91
Length of filtered bad list 12
Test non filtered ['able', 'gala', 'low', 'okay', 'unforgiving', 'sequin', 'curvy', 'ridiculous', 'petite', 'regular', 'stiff', 'extra', 'lumpy', 'back-up', 'thick', 'generous', 'beautiful', 'overall', 'unsewn', 'short', 'pregnant', 'hippy', 'excited', 'lovely', 'needless', 'tight', 'perfect', 'actual', 'form', 'alternative', 'spot-on', 'third', 'alternate', 'single', 'unflattering', 'little', 'heavy', 'super', 'comfortable', 'highlighted', 'reliable', 'fine', 'stylish', 'pretty', 'hanging', 'flimsy', 'v', "5'7", 'white', 'double', 'happy', '34c', 'front', 'boxy', 'bad', 'hard', 'lightweight', 'similar', 'disappointed', '34d', 'shapeless', 'fit', 'true', 'awful', 'online', 'xtra', 'adidas', 'tall', 'unwearable', 'miss', 'bizarre', 'peplum', 'cute', '25in', 'amazing', '18w', 'due', 'thin', 'rouching', "5'8", 'available', 'careful', 'fitting', 'safe', 'odd', 'dated', 'terrible', 'bummer....great', 'disappointing', 'airy', 'una

100%|████████████████████████████████████████████| 1000/1000 [00:04<00:00, 245.49it/s]

Training done


In [221]:
# test output
print(knn.classify("this dress is really lovely. it's got great lines and does amazing things for your bust. however, the two slits have a kind of loin-cloth effect"))
print(knn.classify('beautiful cute dress'))
print(knn.classify('not reccommend unforgiving short lumpy stiff and needless, disappointing awful time'))
print(knn.classify('uncomfortable shapeless unflattering'))

Good
Good
Good
Good
